# Sample Project: 8-puzzle
Note how in the BFS/DFS problems, we didn't need to have access to the entire graph. All we needed was a node to start from, a way to find the children of that node, and a way to determine if a node was a goal (sometimes there is a single goal node, but other times there may be multiple). Thus when we are solving many graph problems, we can build the parts of the graph that we need without necessarily pre-building and storing the entire graph in memory. We don't need a _graph_ so much as:
* a starting state
* a way to compute the children of a given state
* a way to determine if a state is a goal (there may be multiple, acceptable goals in some problems)

A great number of problems can be framed in the context of a search across a graph by defining these three elements. This is useful becomes graph problems are well-characterized and there are already many algorithms for working with them. This exercise gives a fun example of how we can frame a problem as a graph and solve it using search.

If you're not sure what an 8-puzzle is, play one quickly [here][8-puzz]. This problem is much more open-ended and may take more time, but it's also (in my opinion) the most practical; in real-life, you're going to need to find a way to convert your own problems into something that an algorithm can solve. You'll often be using/adapting known algorithms, but much of the difficulty is often just framining the problem correctly. Write a function to solve an 8-puzzle.

---------------

Hints (not too much to say here; if you need more help, you can look at the solution code below)

* You'll probably want to start by defining an 8-puzzle class; you already know search algorithms that you can use to solve this problem once you have the class working properly. From above, you know the three elements you need in such a class. From playing the game a little, it should be clear which states are reachable from a given one (i.e. one slide (one edge) can get you from a given position to which others? Those are the children of that position).
* You need to find a way to store the configuration of the board and to tell what new configurations you can reach from it.
* You'll probably want a decent string representation of your 8-puzzle class so that you can watch the steps of solving it for debugging.
* As I don't know how you'll write your Puzzle class, you'll have to do your own testing for correctness; you can see what I tested at the end of the solution code. Essentially, I gave it a couple of puzzles to start with and checked that it computed the right paths (or found no solution if there was none; be aware that an 8-puzzle is not always solvable. There's a proof about what has to hold for any given state for it to be solvable; I use that because I was aware of it before I tried to code this up, but I don't think you need to).
* If you want to compare with something (though my code hasn't been tested that thoroughly), I've written an example with my code below; you can change the Puzzle to be a simpler one that you're debugging on if needed

[8-puzz]: http://mypuzzle.org/sliding

In [1]:
from collections import deque

In [2]:
class Puzzle(object):
    '''
    Implement a puzzle data structure for the 8-puzzle.

    Your Puzzle class must define initial_state, goal_state, and next_states(s) as
    specified in the problem statement
    '''

    def __init__(self, starting_state):
        """
        __init__(self, starting_state): initializes the Puzzle with the given argument
        as the initial state; must initialize initial_state and goal_state
        ARGS:
          starting_state - the starting state of the puzzle
        """
        self.initial_state = starting_state
        self.goal_state = (1, 2, 3, 4, 5, 6, 7, 8, 0) # 0 denotes the blank space

    def __len__(self):
        return 9

    def __getitem__(self, index):
        return self.initial_state[index] # so you can do Puzzle[i] and get back Puzzle.initial_state[i]

    def __str__(self):
        return str(self.initial_state[:3]) + "\n" + str(self.initial_state[3:6]) + "\n" + str(self.initial_state[6:])
    
    def __repr__(self):
        return self.__str__()

    def swap(self, state, i, j):
        """
        Swaps two elements in the given state tuple.
        :param state: a tuple with the current state of a puzzle
        :type state: tuple
        :param i: index of one element to swap
        :param j: index of other element to swap; j != i
        :return: a tuple which is a copy of state except the elements at i, j are swapped
        :rtype: tuple
        """
        first = min(i, j)
        second = max(i, j)
        swapped = state[:first] + (state[second],) + state[first+1:second] + (state[first],) + state[second+1:]
        # if you make states lists instead of tuples, you can do this more easily, but I wanted them immutable
        return swapped

    def next_states(self, state):
        """
        Returns a list of all the states reachable from the given state with one slide
        :type state: tuple
        :param state: the state from which to
        :rtype: list[tuple]
        :return: a list of states that are reachable with one slide
        """
        next_states = []
        blank_index = state.index(0) # where the blank spot is; slides have to move a piece to here

        # this is the trickiest part. The slides we can do are left, right, up/top, and down/bottom. However, all of these can only be done if the
        # blank is in the center. If the blank is on the left side, we can't slide left; if on the top, we can't slide up; etc.
        # Thus we check where the blank spot is, and each side that it's not on is a valid direction to move
        
        left_indices = (0, 3, 6)
        right_indices = (2, 5, 8)
        top_indices = (0, 1, 2)
        bottom_indices = (6, 7, 8)
        left_swap = -1 # sliding left means the index goes down one
        right_swap = 1 # up one for a right slide
        top_swap = -3 # down 3 for a top slide
        bottom_swap = 3 # up three for a bottom slide

        valid_swaps = []
        for indices, swap_value in zip([left_indices, right_indices, top_indices, bottom_indices], [left_swap, right_swap, top_swap, bottom_swap]):
            if blank_index not in indices: valid_swaps.append(swap_value)

        for swap_value in valid_swaps:
            next_states.append(self.swap(state, blank_index, blank_index + swap_value))

        return next_states


class PuzzleNode(Puzzle):
    """
    Helper class for solving Puzzles. Keeps track of parent PuzzleNode and its own puzzle state as a tuple
    """

    def __init__(self, parent, puzzle_state):
        """
        :param parent:
        :type parent: PuzzleNode
        :param puzzle_state:
        :type puzzle_state: tuple
        :return:
        """
        
        self.parent = parent
        # super(Puzzle, self).__init__(puzzle_state)
        Puzzle.__init__(self, puzzle_state)

#     def __str__(self):
#         if self.parent is None:
#             return "Parent:\n" + "None" + "\nSelf:\n" + Puzzle.__str__(self)
#         else:
#             return "Parent:\n" + Puzzle.__str__(self.parent) + "\nSelf:\n" + Puzzle.__str__(self)

    def get_parent(self):
        return self.parent

    def get_state(self):
        return self.initial_state

    def get_children(self):
        """
        Returns a list of all of the children of the given PuzzleNode
        :return: a list of the children of the given PuzzleNode
        :rtype: list[PuzzleNode]
        """
        return [PuzzleNode(self, state) for state in self.next_states(self.initial_state)]

    def get_path(self):
        """
        :return:
        :rtype: list[tuple]
        """
        if self.parent is None:
            return [self]
        else:
            return self.parent.get_path() + [self]


def solve_puzzle(P):
    """
    Given an 8-puzzle data structure, returns the shortest sequence of states that can be used to solve the puzzle
    
    :param P: the 8-puzzle, with initial_state, goal_state, and next_states(s) defined
    :type P: Puzzle
    :return: the sequence of states used to solve the puzzle in the fewest moves (as a list); if there are no possible solutions,
             return the empty list
    :rtype: list[tuple]
    """
    
    # you should be able to just use your BFS implementation from before as long as you standardize your Puzzle class functions
    # (like get_children and is_goal) with those of your Node class; I wrote this code before writing the Node class or the BFS/DFS
    # functions above; it does the same as my BFS, though, except that for unsolvable Puzzles, it doesn't bother trying
    # (it's not meant to be obvious that the parity of the number of inversions shows whether the Puzzle is solvable; there's a proof
    # of that, however)
    
    puzzle_root = PuzzleNode(None, P.initial_state)
    num_inversions = 0
    for i in range(len(puzzle_root)):
        if puzzle_root[i] == 0: continue  # ignore blank space
        for j in range(i):  # all spaces before this one
            if puzzle_root[j] > puzzle_root[i]:
                num_inversions += 1
    if num_inversions % 2 != 0: return []
    
    # BFS
    nodes_to_process = deque([puzzle_root])
    nodes_visited = set()
    while nodes_to_process:
        node = nodes_to_process.popleft()
        if node.initial_state in nodes_visited:
            continue
        elif node.initial_state == puzzle_root.goal_state:
            return node.get_path()
        else:
            nodes_visited.add(node.initial_state)
            nodes_to_process.extend([child for child in node.get_children() if child.initial_state not in nodes_visited])
    return []  # if this is reached, there was no valid path


def simple_test():
    # Test the Puzzle class and getting next states
    s = (1, 2, 3, 4, 0, 6, 7, 5, 8)
    P = Puzzle(s)
    assert P.initial_state == (1, 2, 3, 4, 0, 6, 7, 5, 8)
    assert P.goal_state == (1, 2, 3, 4, 5, 6, 7, 8, 0)
    s_next_states = P.next_states(s)
    state_solutions = [(1, 0, 3, 4, 2, 6, 7, 5, 8), (1, 2, 3, 0, 4, 6, 7, 5, 8), (1, 2, 3, 4, 6, 0, 7, 5, 8),
                        (1, 2, 3, 4, 5, 6, 7, 0, 8)]
    for i in range(4):
        assert s_next_states[i] in state_solutions

    # Test finding a solution when the Puzzle is and is not solvable
    unsolvable_puzzles = [Puzzle((5, 1, 8, 0, 2, 3, 4, 6, 7))]
    unsolvable_puzzle_solutions = []
    for puzzle in unsolvable_puzzles:
        unsolvable_puzzle_solutions.append(solve_puzzle(puzzle))
    for solution in unsolvable_puzzle_solutions:
        assert solution == []

    solution_path = [(1, 2, 3, 4, 0, 6, 7, 5, 8), (1, 2, 3, 4, 5, 6, 7, 0, 8), (1, 2, 3, 4, 5, 6, 7, 8, 0)]
    solution = solve_puzzle(P)
    for i in range(len(solution)):
        assert solution[i].initial_state == solution_path[i]
    print("Tests passed.")

In [3]:
simple_test()

Tests passed.


In [9]:
s = solution[0]

In [11]:
s

(0, 8, 3)
(6, 4, 7)
(1, 2, 5)

In [13]:
import numpy as np

In [22]:
str(list(map(lambda frame: np.array(frame.get_state()).reshape((3, 3)).tolist(), solution)))

'[[[0, 8, 3], [6, 4, 7], [1, 2, 5]], [[8, 0, 3], [6, 4, 7], [1, 2, 5]], [[8, 4, 3], [6, 0, 7], [1, 2, 5]], [[8, 4, 3], [0, 6, 7], [1, 2, 5]], [[8, 4, 3], [1, 6, 7], [0, 2, 5]], [[8, 4, 3], [1, 6, 7], [2, 0, 5]], [[8, 4, 3], [1, 0, 7], [2, 6, 5]], [[8, 4, 3], [0, 1, 7], [2, 6, 5]], [[0, 4, 3], [8, 1, 7], [2, 6, 5]], [[4, 0, 3], [8, 1, 7], [2, 6, 5]], [[4, 1, 3], [8, 0, 7], [2, 6, 5]], [[4, 1, 3], [8, 7, 0], [2, 6, 5]], [[4, 1, 3], [8, 7, 5], [2, 6, 0]], [[4, 1, 3], [8, 7, 5], [2, 0, 6]], [[4, 1, 3], [8, 7, 5], [0, 2, 6]], [[4, 1, 3], [0, 7, 5], [8, 2, 6]], [[4, 1, 3], [7, 0, 5], [8, 2, 6]], [[4, 1, 3], [7, 2, 5], [8, 0, 6]], [[4, 1, 3], [7, 2, 5], [0, 8, 6]], [[4, 1, 3], [0, 2, 5], [7, 8, 6]], [[0, 1, 3], [4, 2, 5], [7, 8, 6]], [[1, 0, 3], [4, 2, 5], [7, 8, 6]], [[1, 2, 3], [4, 0, 5], [7, 8, 6]], [[1, 2, 3], [4, 5, 0], [7, 8, 6]], [[1, 2, 3], [4, 5, 6], [7, 8, 0]]]'

In [4]:
solution = solve_puzzle(Puzzle((0, 8, 3, 6, 4, 7, 1, 2, 5)))
for state in solution:
    print(state, end='\n\n')

(0, 8, 3)
(6, 4, 7)
(1, 2, 5)

(8, 0, 3)
(6, 4, 7)
(1, 2, 5)

(8, 4, 3)
(6, 0, 7)
(1, 2, 5)

(8, 4, 3)
(0, 6, 7)
(1, 2, 5)

(8, 4, 3)
(1, 6, 7)
(0, 2, 5)

(8, 4, 3)
(1, 6, 7)
(2, 0, 5)

(8, 4, 3)
(1, 0, 7)
(2, 6, 5)

(8, 4, 3)
(0, 1, 7)
(2, 6, 5)

(0, 4, 3)
(8, 1, 7)
(2, 6, 5)

(4, 0, 3)
(8, 1, 7)
(2, 6, 5)

(4, 1, 3)
(8, 0, 7)
(2, 6, 5)

(4, 1, 3)
(8, 7, 0)
(2, 6, 5)

(4, 1, 3)
(8, 7, 5)
(2, 6, 0)

(4, 1, 3)
(8, 7, 5)
(2, 0, 6)

(4, 1, 3)
(8, 7, 5)
(0, 2, 6)

(4, 1, 3)
(0, 7, 5)
(8, 2, 6)

(4, 1, 3)
(7, 0, 5)
(8, 2, 6)

(4, 1, 3)
(7, 2, 5)
(8, 0, 6)

(4, 1, 3)
(7, 2, 5)
(0, 8, 6)

(4, 1, 3)
(0, 2, 5)
(7, 8, 6)

(0, 1, 3)
(4, 2, 5)
(7, 8, 6)

(1, 0, 3)
(4, 2, 5)
(7, 8, 6)

(1, 2, 3)
(4, 0, 5)
(7, 8, 6)

(1, 2, 3)
(4, 5, 0)
(7, 8, 6)

(1, 2, 3)
(4, 5, 6)
(7, 8, 0)

